In [1]:
import os
import re
import sys
from datetime import datetime

game_started = re.compile("((?:\d+\.?){3} \d+:\d+:\d+.\d) : Game started")
game_stopped = re.compile("((?:\d+\.?){3} \d+:\d+:\d+.\d) : Game stopped")
betterlogs = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 💡 BetterLogs: Joined player.* id: (\d+), name: (.*), ip: \d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}, auth: (.+)")
chatline = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : .*#\d+ : .*")
score_limit = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 📢 : (.+)🏆 Лимит по очкам! .*")
time_limit = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 📢 : (.+)🏆 Время вышло.*")

def starting_lineup(match):
    players = {'Red' : [], 'Blue': []}
    game_started = re.compile("(\d+\.?){3} \d+:\d+:\d+.\d : Game started")
    player_side = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : (.*) was moved to (\w{3,4})$")
    for event in match:
        if game_started.match(event):
            break
        if player_side.match(event):
            players[player_side.match(event).group(2)].append(player_side.match(event).group(1))
    return players

def match_result(match):
    winner = 0
    the_event = match[-1]    
    for m in reversed(match):
        if score_limit.match(m) or time_limit.match(m):
            the_event = m
            break
            
    if score_limit.match(the_event):
        if score_limit.match(the_event).group(1) == "🔵":
            winner = 2
        else: 
            winner = 1
    elif time_limit.match(the_event):
        if time_limit.match(the_event).group(1) == "🔵":
            winner = 2
        else: 
            winner = 1
    return winner


def get_goals(match, players):
    goals = {p:0 for p in players}
    for line in match:
        for name in players:
            if '⚽'+name in line and "| Автогол |" not in line: 
                goals[name] += 1
    return goals
                
    
def get_stats(winner, players, m):
    player_stats = {p: {'goals': 0, 'wins' : 0} for p in players['Red'] + players['Blue']}
    if winner > 0 and len(players['Red']) == 4:
        if winner == 1:
            for p in players['Red']:
                player_stats[p]['wins'] += 1
        else: 
            for p in players['Blue']:
                player_stats[p]['wins'] += 1
        
        goals = get_goals(m, players['Red'] + players['Blue'])
        for p in goals.keys():
            player_stats[p]['goals'] += goals[p]
    else:
        return {'status' : '!4x4'}
    
    player_stats_auth = {}
    for k, v in player_stats.items():
        if k in stack_players.keys():
            player_stats_auth[stack_players[k]] = v
    return {'stats': player_stats_auth}
        


def analyse_match():
    match = current_match['logs'] 
    match_players = starting_lineup(match)
    match_winner = match_result(match)
    match_stats = get_stats(match_winner, match_players, match)
    current_match.update(match_stats)    
    
def processing(line):
    global match_id, current_match, db_matches
    
    if not chatline.match(line) and not betterlogs.match(line):
        current_match['logs'].append(line.strip())
        
    if game_stopped.match(line):
        current_match['status'] = 'stopped'
        current_match['stopped_at'] = datetime.strptime(game_stopped.match(line).group(1), 
                                                      '%d.%m.%Y %H:%M:%S.%f')
        analyse_match()
        del(current_match['logs'])
        db_matches.append(current_match)
        match_id += 1
        current_match = {'status' : 'init', 'id' : match_id, 'logs' : []}
    if game_started.match(line):
        current_match['status'] = 'in progress',
        current_match['start_at'] = datetime.strptime(game_started.match(line).group(1), 
                                                      '%d.%m.%Y %H:%M:%S.%f')
    
    if betterlogs.match(line):
        nickname = betterlogs.match(line).group(2).strip()
        auth_key = betterlogs.match(line).group(3)
        stack_players[nickname] = auth_key
        
match_id = 0
current_match = {'status' : '', 'id' : -1, 'logs' : []}
db_matches = []
stack_players = {}

folder = 'dataHAX/all_files/'
for filename in os.listdir(folder):
    f = os.path.join(folder, filename)
    with open(f, 'r', encoding='utf-8') as file:
        for line in file:
            processing(line)

#db_matches

In [7]:
stats = {}
for match in db_matches:
    if match['status'] == 'stopped':
        for auth, match_stats in match['stats'].items():
            if (match['start_at'].strftime('%Y-%m') , auth) not in stats.keys():
                stats[(match['start_at'].strftime('%Y-%m') , auth)] = {
                        'goals' : match_stats['goals'],
                        'matches' : 1,
                        'wins' : match_stats['wins'],
                        'duration' : match['stopped_at'] - match['start_at']
                    }
            else:
                stats[(match['start_at'].strftime('%Y-%m') , auth)]['goals'] += match_stats['goals']
                stats[(match['start_at'].strftime('%Y-%m') , auth)]['matches'] += 1
                stats[(match['start_at'].strftime('%Y-%m') , auth)]['wins'] += match_stats['wins']
                stats[(match['start_at'].strftime('%Y-%m') , auth)]['duration'] += (match['stopped_at'] - match['start_at'])

#stats

In [17]:
def processing_nick(line):
    betterlogs = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 💡 BetterLogs: Joined player.* id: (\d+), name: (.*), ip: \d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}, auth: (.+)")
    if betterlogs.match(line):
        nick = betterlogs.match(line).group(2).strip()
        id = betterlogs.match(line).group(3)
        
        if id not in players.keys():
            players[id] = {nick : 1}
        else:
            if nick in players[id].keys():
                players[id][nick] += 1
            else:
                players[id][nick] = 1

players = {}

folder = 'dataHAX/all_files/'
for filename in os.listdir(folder):
    f = os.path.join(folder, filename)
    # day_nick[filename] = 0
    with open(f, 'r', encoding='utf-8') as file:
        for line in file:
            processing_nick(line)

import operator

users = {}

for k,v in dict(sorted(players.items(), key=lambda item: sum(item[1].values()), reverse=True)).items():
    if max(v.items(), key=operator.itemgetter(1))[0] not in users.keys():
        users[max(v.items(), key=operator.itemgetter(1))[0]] = {
            'idkeys' : [k],
            'visits' : sum(v.values())
        }
    else:
        users[max(v.items(), key=operator.itemgetter(1))[0]]['idkeys'].append(k)
        users[max(v.items(), key=operator.itemgetter(1))[0]]['visits'] += sum(v.values())
    if sum(v.values()) < 10:
        break

#sorted(users.items(), key = lambda item : item[1]['visits'], reverse=True)

users_invert = {}
for nick, data in users.items():
    for idkey in data['idkeys']:
        users_invert[idkey] = nick
        


In [18]:
with open('dataHAX/stats_by_month_4.csv', 'w') as f:
    f.write('date, nickname, goals, matches, wins, spent_h\n')
    for k , v in stats.items():
        try:
            f.write('%s, "%s", %s, %s, %s, %s\n' % (k[0], users_invert[k[1]],
                                              v['goals'], v['matches'],
                                              v['wins'], round(v['duration'].seconds / 3600)
                                             ))
        except:
            pass